In [190]:
"""
解析 mini SWE agent 运行结果的工具函数

目录结构:
run_results/
  task_id_1/
    instance_id_1/
      instance_id_1.traj.json
    instance_id_2/
      instance_id_2.traj.json
    ...
    preds.json
    *_eval.json (例如: openai_gpt-5-mini.mini_eval.json)
  task_id_2/
    ...
"""

import json
from pathlib import Path
from typing import Dict, List, Any, Optional
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)


def load_run_results(run_results_dir: str) -> Dict[str, Dict[str, Dict[str, Any]]]:
    """
    加载 run_results 目录下所有文件，解析成统一格式
    
    Args:
        run_results_dir: run_results 目录的路径
    
    Returns:
        格式为: {
            'task_id': {
                'instance_id': {
                    'traj': [
                        {
                            'role': str, 
                            'content': str,
                            'completion_tokens': int,
                            'prompt_tokens': int,
                            'total_tokens': int
                        }, 
                        ...
                    ],
                    'is_resolved': bool,
                    'patch': str,
                    'cost': float,
                    'api_calls': int
                }
            }
        }
    """
    results = {}
    run_results_path = Path(run_results_dir)
    
    if not run_results_path.exists():
        raise FileNotFoundError(f"目录不存在: {run_results_dir}")
    
    # 遍历所有 task_id 子目录
    for task_dir in run_results_path.iterdir():
        if not task_dir.is_dir() or task_dir.name == '.ipynb_checkpoints':
            continue
            
        task_id = task_dir.name
        logger.info(f"处理 task_id: {task_id}")
        
        # 加载该 task 的 preds.json 和 eval.json
        preds_data = load_preds_json(task_dir)
        eval_data = load_eval_json(task_dir)
        
        results[task_id] = {}
        
        # 遍历所有 instance_id 子目录
        for instance_dir in task_dir.iterdir():
            if not instance_dir.is_dir() or instance_dir.name == '.ipynb_checkpoints':
                continue
                
            instance_id = instance_dir.name
            logger.debug(f"  处理 instance_id: {instance_id}")
            
            # 加载 trajectory，同时获取 cost 和 api_calls
            traj_data = load_trajectory(instance_dir, instance_id)
            traj = traj_data['messages']
            cost = traj_data['cost']
            api_calls = traj_data['api_calls']
            
            # 获取 patch
            patch = preds_data.get(instance_id, {}).get('model_patch', '')
            
            # 获取 is_resolved 状态
            is_resolved = eval_data.get(instance_id, {}).get('resolved', False)
            
            results[task_id][instance_id] = {
                'traj': traj,
                'is_resolved': is_resolved,
                'patch': patch,
                'cost': cost,
                'api_calls': api_calls
            }
    
    logger.info(f"总共加载了 {len(results)} 个 task_id")
    total_instances = sum(len(instances) for instances in results.values())
    logger.info(f"总共加载了 {total_instances} 个 instance")
    
    return results


def load_preds_json(task_dir: Path) -> Dict[str, Any]:
    """
    加载 preds.json 文件
    
    preds.json 格式 (mini SWE agent):
    {
        "instance_id": {
            "model_name_or_path": "...",
            "instance_id": "...",
            "model_patch": "..."
        }
    }
    """
    preds_file = task_dir / 'preds.json'
    
    if not preds_file.exists():
        logger.warning(f"未找到 preds.json: {preds_file}")
        return {}
    
    try:
        with open(preds_file, 'r', encoding='utf-8') as f:
            data = json.load(f)
        logger.debug(f"  加载 preds.json: {len(data)} 个预测")
        return data
    except json.JSONDecodeError as e:
        logger.error(f"解析 preds.json 失败: {preds_file}, 错误: {e}")
        return {}


def load_eval_json(task_dir: Path) -> Dict[str, Any]:
    """
    加载评估结果 JSON 文件 (*_eval.json)
    
    mini-swe-agent 评估结果格式:
    {
        "total_instances": 50,
        "submitted_instances": 50,
        "completed_instances": 46,
        "resolved_instances": 26,
        "unresolved_instances": 20,
        "empty_patch_instances": 2,
        "error_instances": 2,
        "completed_ids": [...],
        "incomplete_ids": [...],
        "empty_patch_ids": [...],
        "submitted_ids": [...],
        "resolved_ids": ["instance_id_1", "instance_id_2", ...]
    }
    
    也支持 SWE-bench 标准格式:
    {
        "instance_id": {
            "FAIL_TO_PASS": {...},
            "PASS_TO_PASS": {...},
            ...
        }
    }
    """
    # 查找所有 *_eval.json 文件
    eval_files = list(task_dir.glob('*_eval.json'))
    
    if not eval_files:
        # 也尝试查找 results.json 或 report.json
        eval_files = list(task_dir.glob('results.json'))
        if not eval_files:
            eval_files = list(task_dir.glob('report.json'))
    
    if not eval_files:
        logger.warning(f"未找到评估结果文件 (*_eval.json, results.json, report.json): {task_dir}")
        return {}
    
    # 使用第一个找到的评估文件
    eval_file = eval_files[0]
    
    try:
        with open(eval_file, 'r', encoding='utf-8') as f:
            data = json.load(f)
        
        logger.debug(f"  加载评估文件: {eval_file.name}")
        
        # 解析评估结果，提取 is_resolved 信息
        parsed_data = {}
        
        # 格式1: mini-swe-agent 格式 - 有 resolved_ids 列表
        if 'resolved_ids' in data:
            resolved_ids = set(data['resolved_ids'])
            # 从 submitted_ids 获取所有实例
            all_ids = data.get('submitted_ids', [])
            for instance_id in all_ids:
                parsed_data[instance_id] = {
                    'resolved': instance_id in resolved_ids
                }
            logger.debug(f"    mini-swe-agent 格式: {len(resolved_ids)} 已解决 / {len(all_ids)} 总计")
            return parsed_data
        
        # 格式2: 每个 instance 都有单独的评估结果
        for instance_id, instance_data in data.items():
            if isinstance(instance_data, dict):
                # 方式2a: 直接有 resolved 字段
                if 'resolved' in instance_data:
                    parsed_data[instance_id] = {'resolved': instance_data['resolved']}
                # 方式2b: 根据 SWE-bench 的 FAIL_TO_PASS 和 PASS_TO_PASS 判断
                elif 'FAIL_TO_PASS' in instance_data or 'PASS_TO_PASS' in instance_data:
                    fail_to_pass = instance_data.get('FAIL_TO_PASS', {})
                    pass_to_pass = instance_data.get('PASS_TO_PASS', {})
                    
                    if isinstance(fail_to_pass, dict):
                        # 检查是否所有测试都通过
                        all_passed = all(v for v in fail_to_pass.values()) if fail_to_pass else False
                        no_regressions = all(v for v in pass_to_pass.values()) if pass_to_pass else True
                        parsed_data[instance_id] = {'resolved': all_passed and no_regressions}
                    else:
                        parsed_data[instance_id] = {'resolved': False}
                # 方式2c: 其他格式，默认为 False
                else:
                    parsed_data[instance_id] = {'resolved': False}
            else:
                parsed_data[instance_id] = {'resolved': False}
        
        logger.debug(f"    解析了 {len(parsed_data)} 个实例的评估结果")
        return parsed_data
        
    except json.JSONDecodeError as e:
        logger.error(f"解析评估文件失败: {eval_file}, 错误: {e}")
        return {}
    except Exception as e:
        logger.error(f"读取评估文件失败: {eval_file}, 错误: {e}")
        return {}


def load_trajectory(instance_dir: Path, instance_id: str) -> Dict[str, Any]:
    """
    加载 trajectory 文件 (*.traj.json)
    
    mini-swe-agent trajectory 格式:
    {
        "info": {...},
        "model_stats": {
            "instance_cost": 0.0068863,
            "api_calls": 8,
            "mini_version": "1.17.3"
        },
        "messages": [
            {
                "role": "system" | "assistant" | "user",
                "content": "...",
                "timestamp": "...",
                "extra": {
                    "response": {
                        "usage": {
                            "input_tokens": 1173,          # prompt tokens
                            "output_tokens": 1143,         # completion tokens
                            "output_tokens_details": {
                                "reasoning_tokens": 960    # 推理 tokens (如 o1/o3)
                            },
                            "total_tokens": 2316,
                            # 兼容旧格式:
                            "prompt_tokens": 1426,
                            "completion_tokens": 172
                        }
                    }
                }
            },
            ...
        ],
        "trajectory_format": "mini-swe-agent-1",
        "instance_id": "..."
    }
    
    返回格式: {
        'messages': [
            {
                'role': str,
                'content': str,
                'completion_tokens': int,
                'prompt_tokens': int,
                'total_tokens': int,
                'reasoning_tokens': int  # 推理 token 数量（如果支持）
            },
            ...
        ],
        'cost': float,
        'api_calls': int
    }
    """
    # 查找 .traj.json 文件
    traj_files = list(instance_dir.glob('*.traj.json'))
    
    if not traj_files:
        logger.warning(f"未找到 trajectory 文件: {instance_dir}")
        return {
            'messages': [],
            'cost': 0.0,
            'api_calls': 0
        }
    
    # 使用第一个找到的 trajectory 文件
    traj_file = traj_files[0]
    
    try:
        with open(traj_file, 'r', encoding='utf-8') as f:
            data = json.load(f)
        
        # 提取 cost 和 api_calls
        # 尝试多个可能的位置：
        # 1. info.model_stats (mini-swe-agent 实际位置)
        # 2. 顶层 model_stats
        # 3. 顶层直接字段
        
        cost = 0.0
        api_calls = 0
        
        # 方法1: 从 info.model_stats 获取（实际位置！）
        if 'info' in data and isinstance(data['info'], dict):
            info = data['info']
            if 'model_stats' in info and isinstance(info['model_stats'], dict):
                model_stats = info['model_stats']
                cost = model_stats.get('instance_cost', model_stats.get('cost', 0.0))
                api_calls = model_stats.get('api_calls', 0)
        
        # 方法2: 从顶层 model_stats 获取（备用）
        if cost == 0.0 and 'model_stats' in data:
            model_stats = data.get('model_stats', {})
            cost = model_stats.get('instance_cost', model_stats.get('cost', 0.0))
            api_calls = model_stats.get('api_calls', 0)
        
        # 方法3: 直接从顶层获取（备用）
        if cost == 0.0 and 'instance_cost' in data:
            cost = data.get('instance_cost', 0.0)
        if cost == 0.0 and 'cost' in data:
            cost = data.get('cost', 0.0)
        if api_calls == 0 and 'api_calls' in data:
            api_calls = data.get('api_calls', 0)
        
        # 调试信息
        if cost == 0.0 and api_calls == 0:
            logger.warning(f"    ⚠️  未找到 cost/api_calls 在 {instance_id}")
            logger.debug(f"    顶层字段: {list(data.keys())}")
            if 'info' in data:
                logger.debug(f"    info 字段: {list(data.get('info', {}).keys())}")
                if 'model_stats' in data.get('info', {}):
                    logger.debug(f"    info.model_stats 字段: {list(data['info']['model_stats'].keys())}")
        else:
            logger.debug(f"    ✓ {instance_id}: cost=${cost:.6f}, api_calls={api_calls}")
        
        # mini-swe-agent 格式：从 messages 字段提取
        if 'messages' in data:
            messages = data['messages']
            conversation = []
            
            for msg in messages:
                if 'role' not in msg or 'content' not in msg:
                    continue
                
                # 构建消息基础字段
                message_dict = {
                    'role': msg['role'],
                    'content': msg['content']
                }
                
                # 提取 token 信息（如果存在）
                completion_tokens = 0
                prompt_tokens = 0
                total_tokens = 0
                reasoning_tokens = 0
                
                if 'extra' in msg and isinstance(msg['extra'], dict):
                    response = msg['extra'].get('response', {})
                    if isinstance(response, dict):
                        usage = response.get('usage', {})
                        if isinstance(usage, dict):
                            # 新格式：input_tokens, output_tokens (OpenAI API 格式)
                            prompt_tokens = usage.get('input_tokens', 0)
                            completion_tokens = usage.get('output_tokens', 0)
                            total_tokens = usage.get('total_tokens', 0)
                            
                            # 向后兼容旧格式 (completion_tokens, prompt_tokens)
                            if prompt_tokens == 0:
                                prompt_tokens = usage.get('prompt_tokens', 0)
                            if completion_tokens == 0:
                                completion_tokens = usage.get('completion_tokens', 0)
                            
                            # 提取 reasoning_tokens（位于 output_tokens_details 中）
                            output_tokens_details = usage.get('output_tokens_details', {})
                            if isinstance(output_tokens_details, dict):
                                reasoning_tokens = output_tokens_details.get('reasoning_tokens', 0)
                
                message_dict['completion_tokens'] = completion_tokens
                message_dict['prompt_tokens'] = prompt_tokens
                message_dict['total_tokens'] = total_tokens
                message_dict['reasoning_tokens'] = reasoning_tokens
                
                conversation.append(message_dict)
            
            logger.debug(f"    加载 trajectory (mini-swe-agent): {len(conversation)} 条消息, cost={cost}, api_calls={api_calls}")
            
            return {
                'messages': conversation,
                'cost': cost,
                'api_calls': api_calls
            }
        
        # 兼容旧的 SWE-agent 格式
        elif 'trajectory' in data:
            trajectory = data['trajectory']
            conversation = []
            
            for step in trajectory:
                if 'thought' in step and step['thought']:
                    conversation.append({
                        'role': 'assistant',
                        'content': f"[THOUGHT] {step['thought']}",
                        'completion_tokens': 0,
                        'prompt_tokens': 0,
                        'total_tokens': 0,
                        'reasoning_tokens': 0
                    })
                
                if 'action' in step and step['action']:
                    conversation.append({
                        'role': 'assistant',
                        'content': f"[ACTION] {step['action']}",
                        'completion_tokens': 0,
                        'prompt_tokens': 0,
                        'total_tokens': 0,
                        'reasoning_tokens': 0
                    })
                
                if 'observation' in step and step['observation']:
                    conversation.append({
                        'role': 'user',
                        'content': f"[OBSERVATION] {step['observation']}",
                        'completion_tokens': 0,
                        'prompt_tokens': 0,
                        'total_tokens': 0,
                        'reasoning_tokens': 0
                    })
                
                if 'response' in step and step['response']:
                    if 'thought' not in step and 'action' not in step:
                        conversation.append({
                            'role': 'assistant',
                            'content': step['response'],
                            'completion_tokens': 0,
                            'prompt_tokens': 0,
                            'total_tokens': 0,
                            'reasoning_tokens': 0
                        })
            
            logger.debug(f"    加载 trajectory (swe-agent): {len(conversation)} 条消息")
            
            return {
                'messages': conversation,
                'cost': 0.0,
                'api_calls': 0
            }
        
        else:
            logger.warning(f"无法识别的 trajectory 格式: {traj_file}")
            return {
                'messages': [],
                'cost': 0.0,
                'api_calls': 0
            }
        
    except json.JSONDecodeError as e:
        logger.error(f"解析 trajectory 文件失败: {traj_file}, 错误: {e}")
        return {
            'messages': [],
            'cost': 0.0,
            'api_calls': 0
        }
    except Exception as e:
        logger.error(f"读取 trajectory 文件失败: {traj_file}, 错误: {e}")
        return {
            'messages': [],
            'cost': 0.0,
            'api_calls': 0
        }

def save_results(results: Dict, output_file: str):
    """保存解析结果到 JSON 文件"""
    output_path = Path(output_file)
    
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(results, f, indent=2, ensure_ascii=False)
    
    logger.info(f"结果已保存到: {output_file}")


def print_summary(results: Dict):
    """打印结果摘要"""
    total_tasks = len(results)
    total_instances = sum(len(instances) for instances in results.values())
    total_resolved = sum(
        sum(1 for inst in instances.values() if inst['is_resolved'])
        for instances in results.values()
    )
    total_cost = sum(
        sum(inst['cost'] for inst in instances.values())
        for instances in results.values()
    )
    total_api_calls = sum(
        sum(inst['api_calls'] for inst in instances.values())
        for instances in results.values()
    )
    
    print("\n" + "="*60)
    print("解析结果摘要")
    print("="*60)
    print(f"总 task 数: {total_tasks}")
    print(f"总 instance 数: {total_instances}")
    print(f"已解决的 instance 数: {total_resolved}")
    if total_instances > 0:
        print(f"解决率: {total_resolved/total_instances*100:.2f}%")
    print(f"\n总 API 调用次数: {total_api_calls}")
    print(f"总成本: ${total_cost:.4f}")
    if total_instances > 0:
        print(f"平均每个 instance 的成本: ${total_cost/total_instances:.4f}")
        print(f"平均每个 instance 的 API 调用: {total_api_calls/total_instances:.1f}")
    print("="*60)


def analyze_by_task(results: Dict, output_file: Optional[str] = None) -> Dict[str, List[Dict]]:
    """
    按 task_id 分析，输出每个 instance 的 cost、api_calls 和 is_resolved
    
    Args:
        results: load_run_results 返回的结果
        output_file: 可选，保存结果到 JSON 文件
    
    Returns:
        格式为: {
            'task_id': [
                {'instance_id': str, 'cost': float, 'api_calls': int, 'is_resolved': bool},
                ...
            ]
        }
    """
    analysis = {}
    
    for task_id, instances in results.items():
        task_analysis = []
        
        for instance_id, data in instances.items():
            task_analysis.append({
                'instance_id': instance_id,
                'cost': data['cost'],
                'api_calls': data['api_calls'],
                'is_resolved': data['is_resolved']
            })
        
        # 按 cost 降序排序
        task_analysis.sort(key=lambda x: x['cost'], reverse=True)
        analysis[task_id] = task_analysis
        
        # 打印该 task 的分析
        print(f"\n{'='*70}")
        print(f"Task ID: {task_id}")
        print(f"{'='*70}")
        print(f"总 instances: {len(task_analysis)}")
        
        # 统计
        resolved = sum(1 for item in task_analysis if item['is_resolved'])
        total_cost = sum(item['cost'] for item in task_analysis)
        total_api_calls = sum(item['api_calls'] for item in task_analysis)
        
        print(f"已解决: {resolved} / {len(task_analysis)} ({resolved/len(task_analysis)*100:.1f}%)")
        print(f"总成本: ${total_cost:.4f}")
        print(f"总 API 调用: {total_api_calls}")
        
        if task_analysis:
            avg_cost = total_cost / len(task_analysis)
            avg_api = total_api_calls / len(task_analysis)
            print(f"平均成本: ${avg_cost:.4f}")
            print(f"平均 API 调用: {avg_api:.1f}")
        
        # 打印前 10 个最贵的 instances
        print(f"\n{'Instance ID':<40} {'Cost':>10} {'API':>6} {'Resolved':>10}")
        print("-" * 70)
        for item in task_analysis[:10]:
            status = "✓" if item['is_resolved'] else "✗"
            print(f"{item['instance_id']:<40} ${item['cost']:>9.4f} {item['api_calls']:>6} {status:>10}")
        
        if len(task_analysis) > 10:
            print(f"... 还有 {len(task_analysis) - 10} 个 instances")
    
    # 保存到文件
    if output_file:
        with open(output_file, 'w', encoding='utf-8') as f:
            json.dump(analysis, f, indent=2, ensure_ascii=False)
        print(f"\n✓ 分析结果已保存到: {output_file}")
    
    return analysis


def get_task_statistics(results: Dict) -> Dict[str, Dict]:
    """
    获取每个 task 的详细统计信息
    
    Returns:
        格式为: {
            'task_id': {
                'total_instances': int,
                'resolved': int,
                'unresolved': int,
                'resolution_rate': float,
                'total_cost': float,
                'avg_cost': float,
                'total_api_calls': int,
                'avg_api_calls': float,
                'cost_per_resolved': float,
                'instances': [...]
            }
        }
    """
    stats = {}
    
    for task_id, instances in results.items():
        instance_list = []
        total_cost = 0
        total_api_calls = 0
        resolved_count = 0
        resolved_cost = 0
        
        for instance_id, data in instances.items():
            instance_list.append({
                'instance_id': instance_id,
                'cost': data['cost'],
                'api_calls': data['api_calls'],
                'is_resolved': data['is_resolved']
            })
            
            total_cost += data['cost']
            total_api_calls += data['api_calls']
            
            if data['is_resolved']:
                resolved_count += 1
                resolved_cost += data['cost']
        
        total = len(instances)
        
        stats[task_id] = {
            'total_instances': total,
            'resolved': resolved_count,
            'unresolved': total - resolved_count,
            'resolution_rate': resolved_count / total if total > 0 else 0,
            'total_cost': total_cost,
            'avg_cost': total_cost / total if total > 0 else 0,
            'total_api_calls': total_api_calls,
            'avg_api_calls': total_api_calls / total if total > 0 else 0,
            'cost_per_resolved': resolved_cost / resolved_count if resolved_count > 0 else 0,
            'instances': instance_list
        }
    
    return stats


def print_task_instance_details(results: Dict, task_id: str = None):
    """
    为每个 task 打印所有 instance 的详细信息
    
    输出格式: [instance: {cost, api_calls, is_resolved}]
    
    Args:
        results: 解析后的结果字典
        task_id: 可选，只打印指定的 task。如果为 None，打印所有 task
    """
    # 如果指定了 task_id，只处理该 task
    tasks_to_print = {task_id: results[task_id]} if task_id and task_id in results else results
    
    for tid, instances in tasks_to_print.items():
        print("\n" + "="*80)
        print(f"Task ID: {tid}")
        print("="*80)
        
        # 收集所有 instance 的信息
        instance_details = []
        for instance_id, data in instances.items():
            instance_details.append({
                'instance': instance_id,
                'cost': data['cost'],
                'api_calls': data['api_calls'],
                'is_resolved': data['is_resolved']
            })
        
        # 打印列表格式
        print(f"\nInstances ({len(instance_details)} total):")
        print("[")
        for detail in instance_details:
            resolved_mark = "✓" if detail['is_resolved'] else "✗"
            print(f"  {resolved_mark} {detail['instance']}: {{")
            print(f"      cost: ${detail['cost']:.6f},")
            print(f"      api_calls: {detail['api_calls']},")
            print(f"      is_resolved: {detail['is_resolved']}")
            print(f"  }},")
        print("]")
        
        # 打印统计摘要
        total_cost = sum(d['cost'] for d in instance_details)
        total_api_calls = sum(d['api_calls'] for d in instance_details)
        resolved_count = sum(1 for d in instance_details if d['is_resolved'])
        
        print(f"\n摘要:")
        print(f"  总 instances: {len(instance_details)}")
        print(f"  已解决: {resolved_count} ({resolved_count/len(instance_details)*100:.1f}%)")
        print(f"  未解决: {len(instance_details) - resolved_count}")
        print(f"  总成本: ${total_cost:.6f}")
        print(f"  总 API 调用: {total_api_calls}")
        print(f"  平均成本: ${total_cost/len(instance_details):.6f}")
        print(f"  平均 API 调用: {total_api_calls/len(instance_details):.1f}")


def print_all_tasks_summary(results: Dict):
    """
    打印所有 task 的简要摘要（不包括详细的 instance 列表）
    """
    print("\n" + "="*80)
    print("所有 Tasks 摘要")
    print("="*80)
    
    for task_id, instances in results.items():
        total_cost = sum(d['cost'] for d in instances.values())
        total_api_calls = sum(d['api_calls'] for d in instances.values())
        resolved_count = sum(1 for d in instances.values() if d['is_resolved'])
        total_instances = len(instances)
        
        print(f"\n{task_id}:")
        print(f"  Instances: {total_instances}")
        print(f"  已解决: {resolved_count} ({resolved_count/total_instances*100:.1f}%)")
        print(f"  总成本: ${total_cost:.6f}")
        print(f"  总 API 调用: {total_api_calls}")
        print(f"  平均成本: ${total_cost/total_instances:.6f}")
        print(f"  平均 API 调用: {total_api_calls/total_instances:.1f}")

In [191]:
results = load_run_results('../run_results/') 
# 打印摘要
print_all_tasks_summary(results)

INFO:__main__:处理 task_id: cam-mini-reasoning-high
INFO:__main__:处理 task_id: cam-mini-reasoning-low
INFO:__main__:处理 task_id: cam-mini-reasoning-medium-res-api
INFO:__main__:处理 task_id: cam-mini-reasoning-medium-test2
INFO:__main__:处理 task_id: cam-mini-reasoning-medium-test3
INFO:__main__:处理 task_id: cam-output
INFO:__main__:处理 task_id: cam-test2-output
INFO:__main__:处理 task_id: cam-test3-output
INFO:__main__:处理 task_id: cam-test4-output
INFO:__main__:处理 task_id: cam-test5-output
INFO:__main__:处理 task_id: cam-test6-output
INFO:__main__:处理 task_id: gpt5-cam-output
INFO:__main__:处理 task_id: gpt5-output
INFO:__main__:处理 task_id: mini-first-high-reasoning-12
INFO:__main__:处理 task_id: mini-first-high-reasoning-4
INFO:__main__:处理 task_id: mini-first-high-reasoning-8
INFO:__main__:处理 task_id: mini-output
INFO:__main__:处理 task_id: mini-reasoning-high-res-api
INFO:__main__:处理 task_id: mini-reasoning-low-res-api
INFO:__main__:处理 task_id: mini-reasoning-medium-res-api
INFO:__main__:处理 task_id: min


所有 Tasks 摘要

cam-mini-reasoning-high:
  Instances: 50
  已解决: 30 (60.0%)
  总成本: $2.975291
  总 API 调用: 447
  平均成本: $0.059506
  平均 API 调用: 8.9

cam-mini-reasoning-low:
  Instances: 50
  已解决: 22 (44.0%)
  总成本: $0.289197
  总 API 调用: 312
  平均成本: $0.005784
  平均 API 调用: 6.2

cam-mini-reasoning-medium-res-api:
  Instances: 50
  已解决: 25 (50.0%)
  总成本: $1.098675
  总 API 调用: 552
  平均成本: $0.021973
  平均 API 调用: 11.0

cam-mini-reasoning-medium-test2:
  Instances: 50
  已解决: 21 (42.0%)
  总成本: $0.978333
  总 API 调用: 435
  平均成本: $0.019567
  平均 API 调用: 8.7

cam-mini-reasoning-medium-test3:
  Instances: 50
  已解决: 26 (52.0%)
  总成本: $0.931198
  总 API 调用: 441
  平均成本: $0.018624
  平均 API 调用: 8.8

cam-output:
  Instances: 50
  已解决: 30 (60.0%)
  总成本: $1.494163
  总 API 调用: 926
  平均成本: $0.029883
  平均 API 调用: 18.5

cam-test2-output:
  Instances: 50
  已解决: 26 (52.0%)
  总成本: $1.389008
  总 API 调用: 917
  平均成本: $0.027780
  平均 API 调用: 18.3

cam-test3-output:
  Instances: 50
  已解决: 28 (56.0%)
  总成本: $1.372270
  总 API 调用: 8

In [33]:
results['cam-output']['django__django-11790']['traj'][0].keys()

dict_keys(['role', 'content', 'completion_tokens', 'prompt_tokens', 'total_tokens'])

In [49]:
# [{i:(v['cost'], v['api_calls'], v['is_resolved'])} for i,v in results['mini-output'].items()]

In [52]:
def analyze_resolution_api_correlation(logs_dict):
    """
    分析各个task_id中is_resolved与api_calls和cost的相关性
    
    参数:
        logs_dict: 格式为 {'task_id': {'instance_id': {'is_resolved': bool, 'api_calls': int, 'cost':float...}}}
    """
    import statistics
    
    print("="*70)
    print("各Task ID的 is_resolved 与 api_calls、cost 相关性分析")
    print("="*70)
    
    for task_id, instances in logs_dict.items():
        print(f"\n【{task_id}】")
        print("-" * 70)
        
        # 分别收集已解决和未解决的api_calls和cost
        resolved_api_calls = []
        unresolved_api_calls = []
        all_api_calls = []
        all_resolved = []
        
        resolved_costs = []
        unresolved_costs = []
        all_costs = []
        
        for instance_id, data in instances.items():
            api_calls = data.get('api_calls', 0)
            cost = data.get('cost', 0.0)
            is_resolved = data.get('is_resolved', False)
            
            all_api_calls.append(api_calls)
            all_costs.append(cost)
            all_resolved.append(1 if is_resolved else 0)
            
            if is_resolved:
                resolved_api_calls.append(api_calls)
                resolved_costs.append(cost)
            else:
                unresolved_api_calls.append(api_calls)
                unresolved_costs.append(cost)
        
        # 基本统计
        total = len(instances)
        resolved_count = len(resolved_api_calls)
        unresolved_count = len(unresolved_api_calls)
        
        print(f"总样本数: {total}")
        print(f"  - 已解决: {resolved_count} ({resolved_count/total*100:.1f}%)")
        print(f"  - 未解决: {unresolved_count} ({unresolved_count/total*100:.1f}%)")
        
        # ==================== API调用分析 ====================
        print(f"\n{'='*70}")
        print(f"API 调用分析")
        print(f"{'='*70}")
        
        # 统计已解决的API调用
        if resolved_api_calls:
            print(f"\n已解决问题的 API 调用:")
            print(f"  - 平均值: {statistics.mean(resolved_api_calls):.2f}")
            print(f"  - 中位数: {statistics.median(resolved_api_calls):.2f}")
            print(f"  - 最小值: {min(resolved_api_calls)}")
            print(f"  - 最大值: {max(resolved_api_calls)}")
            if len(resolved_api_calls) > 1:
                print(f"  - 标准差: {statistics.stdev(resolved_api_calls):.2f}")
        
        # 统计未解决的API调用
        if unresolved_api_calls:
            print(f"\n未解决问题的 API 调用:")
            print(f"  - 平均值: {statistics.mean(unresolved_api_calls):.2f}")
            print(f"  - 中位数: {statistics.median(unresolved_api_calls):.2f}")
            print(f"  - 最小值: {min(unresolved_api_calls)}")
            print(f"  - 最大值: {max(unresolved_api_calls)}")
            if len(unresolved_api_calls) > 1:
                print(f"  - 标准差: {statistics.stdev(unresolved_api_calls):.2f}")
        
        # 对比分析
        if resolved_api_calls and unresolved_api_calls:
            avg_resolved = statistics.mean(resolved_api_calls)
            avg_unresolved = statistics.mean(unresolved_api_calls)
            diff = avg_resolved - avg_unresolved
            diff_pct = (diff / avg_unresolved * 100) if avg_unresolved > 0 else 0
            
            print(f"\n对比:")
            print(f"  - 已解决平均 vs 未解决平均: {avg_resolved:.2f} vs {avg_unresolved:.2f}")
            print(f"  - 差异: {diff:+.2f} ({diff_pct:+.1f}%)")
            
            if avg_resolved > avg_unresolved:
                print(f"  ⚠️  已解决问题平均需要更多API调用")
            elif avg_resolved < avg_unresolved:
                print(f"  ✓  已解决问题平均需要更少API调用")
            else:
                print(f"  →  API调用数相近")
        
        # 计算Pearson相关系数
        if len(all_api_calls) > 1 and len(set(all_api_calls)) > 1:
            try:
                # 简单的Pearson相关系数计算
                n = len(all_api_calls)
                sum_x = sum(all_api_calls)
                sum_y = sum(all_resolved)
                sum_xy = sum(x*y for x, y in zip(all_api_calls, all_resolved))
                sum_x2 = sum(x*x for x in all_api_calls)
                sum_y2 = sum(y*y for y in all_resolved)
                
                numerator = n * sum_xy - sum_x * sum_y
                denominator = ((n * sum_x2 - sum_x**2) * (n * sum_y2 - sum_y**2)) ** 0.5
                
                if denominator != 0:
                    correlation = numerator / denominator
                    print(f"\nPearson相关系数: {correlation:.4f}")
                    
                    if abs(correlation) < 0.1:
                        print(f"  → 几乎无相关性")
                    elif abs(correlation) < 0.3:
                        print(f"  → 弱相关")
                    elif abs(correlation) < 0.7:
                        print(f"  → 中等相关")
                    else:
                        print(f"  → 强相关")
                    
                    if correlation > 0:
                        print(f"  → 正相关：API调用越多，越可能解决")
                    elif correlation < 0:
                        print(f"  → 负相关：API调用越多，越不可能解决")
            except:
                pass
        
        # API调用区间分析
        if all_api_calls:
            print(f"\nAPI调用区间分析:")
            max_calls = max(all_api_calls)
            bins = [0, max_calls//4, max_calls//2, max_calls*3//4, max_calls+1]
            
            for i in range(len(bins)-1):
                lower, upper = bins[i], bins[i+1]
                in_range = [
                    data.get('is_resolved', False) 
                    for inst_id, data in instances.items() 
                    if lower <= data.get('api_calls', 0) < upper
                ]
                if in_range:
                    resolved_in_range = sum(in_range)
                    total_in_range = len(in_range)
                    rate = resolved_in_range / total_in_range * 100
                    print(f"  [{lower:3d}, {upper:3d}): {resolved_in_range:2d}/{total_in_range:2d} 解决 ({rate:5.1f}%)")
        
        # ==================== Cost分析 ====================
        print(f"\n{'='*70}")
        print(f"Cost 分析")
        print(f"{'='*70}")
        
        # 统计已解决的Cost
        if resolved_costs:
            print(f"\n已解决问题的 Cost:")
            print(f"  - 平均值: {statistics.mean(resolved_costs):.4f}")
            print(f"  - 中位数: {statistics.median(resolved_costs):.4f}")
            print(f"  - 最小值: {min(resolved_costs):.4f}")
            print(f"  - 最大值: {max(resolved_costs):.4f}")
            if len(resolved_costs) > 1:
                print(f"  - 标准差: {statistics.stdev(resolved_costs):.4f}")
        
        # 统计未解决的Cost
        if unresolved_costs:
            print(f"\n未解决问题的 Cost:")
            print(f"  - 平均值: {statistics.mean(unresolved_costs):.4f}")
            print(f"  - 中位数: {statistics.median(unresolved_costs):.4f}")
            print(f"  - 最小值: {min(unresolved_costs):.4f}")
            print(f"  - 最大值: {max(unresolved_costs):.4f}")
            if len(unresolved_costs) > 1:
                print(f"  - 标准差: {statistics.stdev(unresolved_costs):.4f}")
        
        # 对比分析
        if resolved_costs and unresolved_costs:
            avg_resolved_cost = statistics.mean(resolved_costs)
            avg_unresolved_cost = statistics.mean(unresolved_costs)
            diff_cost = avg_resolved_cost - avg_unresolved_cost
            diff_pct_cost = (diff_cost / avg_unresolved_cost * 100) if avg_unresolved_cost > 0 else 0
            
            print(f"\n对比:")
            print(f"  - 已解决平均 vs 未解决平均: {avg_resolved_cost:.4f} vs {avg_unresolved_cost:.4f}")
            print(f"  - 差异: {diff_cost:+.4f} ({diff_pct_cost:+.1f}%)")
            
            if avg_resolved_cost > avg_unresolved_cost:
                print(f"  ⚠️  已解决问题平均需要更高成本")
            elif avg_resolved_cost < avg_unresolved_cost:
                print(f"  ✓  已解决问题平均需要更低成本")
            else:
                print(f"  →  成本相近")
        
        # 计算Pearson相关系数
        if len(all_costs) > 1 and len(set(all_costs)) > 1:
            try:
                # 简单的Pearson相关系数计算
                n = len(all_costs)
                sum_x = sum(all_costs)
                sum_y = sum(all_resolved)
                sum_xy = sum(x*y for x, y in zip(all_costs, all_resolved))
                sum_x2 = sum(x*x for x in all_costs)
                sum_y2 = sum(y*y for y in all_resolved)
                
                numerator = n * sum_xy - sum_x * sum_y
                denominator = ((n * sum_x2 - sum_x**2) * (n * sum_y2 - sum_y**2)) ** 0.5
                
                if denominator != 0:
                    correlation = numerator / denominator
                    print(f"\nPearson相关系数: {correlation:.4f}")
                    
                    if abs(correlation) < 0.1:
                        print(f"  → 几乎无相关性")
                    elif abs(correlation) < 0.3:
                        print(f"  → 弱相关")
                    elif abs(correlation) < 0.7:
                        print(f"  → 中等相关")
                    else:
                        print(f"  → 强相关")
                    
                    if correlation > 0:
                        print(f"  → 正相关：成本越高，越可能解决")
                    elif correlation < 0:
                        print(f"  → 负相关：成本越高，越不可能解决")
            except:
                pass
        
        # Cost区间分析
        if all_costs:
            print(f"\nCost区间分析:")
            max_cost = max(all_costs)
            bins = [0, max_cost/4, max_cost/2, max_cost*3/4, max_cost+0.0001]
            
            for i in range(len(bins)-1):
                lower, upper = bins[i], bins[i+1]
                in_range = [
                    data.get('is_resolved', False) 
                    for inst_id, data in instances.items() 
                    if lower <= data.get('cost', 0.0) < upper
                ]
                if in_range:
                    resolved_in_range = sum(in_range)
                    total_in_range = len(in_range)
                    rate = resolved_in_range / total_in_range * 100
                    print(f"  [{lower:7.4f}, {upper:7.4f}): {resolved_in_range:2d}/{total_in_range:2d} 解决 ({rate:5.1f}%)")
    
    print("\n" + "="*70)

In [53]:
analyze_resolution_api_correlation(results)

各Task ID的 is_resolved 与 api_calls、cost 相关性分析

【cam-output】
----------------------------------------------------------------------
总样本数: 50
  - 已解决: 30 (60.0%)
  - 未解决: 20 (40.0%)

API 调用分析

已解决问题的 API 调用:
  - 平均值: 13.40
  - 中位数: 12.00
  - 最小值: 5
  - 最大值: 46
  - 标准差: 8.28

未解决问题的 API 调用:
  - 平均值: 26.20
  - 中位数: 20.50
  - 最小值: 10
  - 最大值: 69
  - 标准差: 15.41

对比:
  - 已解决平均 vs 未解决平均: 13.40 vs 26.20
  - 差异: -12.80 (-48.9%)
  ✓  已解决问题平均需要更少API调用

Pearson相关系数: -0.4820
  → 中等相关
  → 负相关：API调用越多，越不可能解决

API调用区间分析:
  [  0,  17): 20/26 解决 ( 76.9%)
  [ 17,  34):  9/18 解决 ( 50.0%)
  [ 34,  51):  1/ 5 解决 ( 20.0%)
  [ 51,  70):  0/ 1 解决 (  0.0%)

Cost 分析

已解决问题的 Cost:
  - 平均值: 0.0185
  - 中位数: 0.0135
  - 最小值: 0.0046
  - 最大值: 0.0994
  - 标准差: 0.0172

未解决问题的 Cost:
  - 平均值: 0.0469
  - 中位数: 0.0334
  - 最小值: 0.0101
  - 最大值: 0.2012
  - 标准差: 0.0427

对比:
  - 已解决平均 vs 未解决平均: 0.0185 vs 0.0469
  - 差异: -0.0284 (-60.5%)
  ✓  已解决问题平均需要更低成本

Pearson相关系数: -0.4281
  → 中等相关
  → 负相关：成本越高，越不可能解决

Cost区间分析:
  [ 0.0000,  0.050

In [91]:
def analyze_cost_statistics(logs_dict):
    """
    分析各个task_id的cost统计，包括总体、解决/未解决状态、以及每次API调用的成本
    
    参数:
        logs_dict: 格式为 {'task_id': {'instance_id': {'is_resolved': bool, 'cost': float, 'api_calls': int, ...}}}
    """
    import statistics
    
    print("="*80)
    print("各Task ID的 Cost 统计分析")
    print("="*80)
    
    for task_id, instances in logs_dict.items():
        print(f"\n【{task_id}】")
        print("-" * 80)
        
        # 收集数据
        all_costs = []
        resolved_costs = []
        unresolved_costs = []
        cost_per_call_all = []
        cost_per_call_resolved = []
        cost_per_call_unresolved = []
        
        for instance_id, data in instances.items():
            cost = data.get('cost', 0)
            api_calls = data.get('api_calls', 0)
            is_resolved = data.get('is_resolved', False)
            
            all_costs.append(cost)
            
            if is_resolved:
                resolved_costs.append(cost)
                if api_calls > 0:
                    cost_per_call_resolved.append(cost / api_calls)
            else:
                unresolved_costs.append(cost)
                if api_calls > 0:
                    cost_per_call_unresolved.append(cost / api_calls)
            
            if api_calls > 0:
                cost_per_call_all.append(cost / api_calls)
        
        # ========== 总体统计 ==========
        print(f"\n📊 总体 Cost 统计 (样本数: {len(all_costs)})")
        if all_costs:
            print(f"  总成本:    ${sum(all_costs):.4f}")
            print(f"  平均值:    ${statistics.mean(all_costs):.4f}")
            print(f"  中位数:    ${statistics.median(all_costs):.4f}")
            print(f"  最小值:    ${min(all_costs):.4f}")
            print(f"  最大值:    ${max(all_costs):.4f}")
            if len(all_costs) > 1:
                print(f"  标准差:    ${statistics.stdev(all_costs):.4f}")
                q1 = statistics.quantiles(all_costs, n=4)[0]
                q3 = statistics.quantiles(all_costs, n=4)[2]
                print(f"  25%分位:   ${q1:.4f}")
                print(f"  75%分位:   ${q3:.4f}")
        
        # ========== 已解决问题的Cost统计 ==========
        print(f"\n✅ 已解决问题 Cost 统计 (样本数: {len(resolved_costs)})")
        if resolved_costs:
            print(f"  总成本:    ${sum(resolved_costs):.4f}")
            print(f"  平均值:    ${statistics.mean(resolved_costs):.4f}")
            print(f"  中位数:    ${statistics.median(resolved_costs):.4f}")
            print(f"  最小值:    ${min(resolved_costs):.4f}")
            print(f"  最大值:    ${max(resolved_costs):.4f}")
            if len(resolved_costs) > 1:
                print(f"  标准差:    ${statistics.stdev(resolved_costs):.4f}")
        else:
            print(f"  (无已解决问题)")
        
        # ========== 未解决问题的Cost统计 ==========
        print(f"\n❌ 未解决问题 Cost 统计 (样本数: {len(unresolved_costs)})")
        if unresolved_costs:
            print(f"  总成本:    ${sum(unresolved_costs):.4f}")
            print(f"  平均值:    ${statistics.mean(unresolved_costs):.4f}")
            print(f"  中位数:    ${statistics.median(unresolved_costs):.4f}")
            print(f"  最小值:    ${min(unresolved_costs):.4f}")
            print(f"  最大值:    ${max(unresolved_costs):.4f}")
            if len(unresolved_costs) > 1:
                print(f"  标准差:    ${statistics.stdev(unresolved_costs):.4f}")
        else:
            print(f"  (无未解决问题)")
        
        # ========== 已解决 vs 未解决对比 ==========
        if resolved_costs and unresolved_costs:
            print(f"\n⚖️  已解决 vs 未解决 对比:")
            avg_resolved = statistics.mean(resolved_costs)
            avg_unresolved = statistics.mean(unresolved_costs)
            diff = avg_resolved - avg_unresolved
            diff_pct = (diff / avg_unresolved * 100) if avg_unresolved > 0 else 0
            
            print(f"  平均成本:  ${avg_resolved:.4f} vs ${avg_unresolved:.4f}")
            print(f"  差异:      ${diff:+.4f} ({diff_pct:+.1f}%)")
            
            total_resolved = sum(resolved_costs)
            total_unresolved = sum(unresolved_costs)
            print(f"  总成本:    ${total_resolved:.4f} vs ${total_unresolved:.4f}")
            
            if avg_resolved > avg_unresolved:
                print(f"  💰 解决问题平均需要更高成本")
            elif avg_resolved < avg_unresolved:
                print(f"  💡 解决问题平均成本更低")
        
        # ========== 每次API调用的Cost统计 ==========
        print(f"\n🔄 每次 API 调用的平均 Cost 统计")
        
        if cost_per_call_all:
            print(f"\n  总体 (样本数: {len(cost_per_call_all)}):")
            print(f"    平均值:    ${statistics.mean(cost_per_call_all):.6f}")
            print(f"    中位数:    ${statistics.median(cost_per_call_all):.6f}")
            print(f"    最小值:    ${min(cost_per_call_all):.6f}")
            print(f"    最大值:    ${max(cost_per_call_all):.6f}")
            if len(cost_per_call_all) > 1:
                print(f"    标准差:    ${statistics.stdev(cost_per_call_all):.6f}")
        
        if cost_per_call_resolved:
            print(f"\n  已解决问题 (样本数: {len(cost_per_call_resolved)}):")
            print(f"    平均值:    ${statistics.mean(cost_per_call_resolved):.6f}")
            print(f"    中位数:    ${statistics.median(cost_per_call_resolved):.6f}")
            print(f"    最小值:    ${min(cost_per_call_resolved):.6f}")
            print(f"    最大值:    ${max(cost_per_call_resolved):.6f}")
        
        if cost_per_call_unresolved:
            print(f"\n  未解决问题 (样本数: {len(cost_per_call_unresolved)}):")
            print(f"    平均值:    ${statistics.mean(cost_per_call_unresolved):.6f}")
            print(f"    中位数:    ${statistics.median(cost_per_call_unresolved):.6f}")
            print(f"    最小值:    ${min(cost_per_call_unresolved):.6f}")
            print(f"    最大值:    ${max(cost_per_call_unresolved):.6f}")
        
        if cost_per_call_resolved and cost_per_call_unresolved:
            avg_per_call_resolved = statistics.mean(cost_per_call_resolved)
            avg_per_call_unresolved = statistics.mean(cost_per_call_unresolved)
            diff_per_call = avg_per_call_resolved - avg_per_call_unresolved
            diff_per_call_pct = (diff_per_call / avg_per_call_unresolved * 100) if avg_per_call_unresolved > 0 else 0
            
            print(f"\n  对比:")
            print(f"    已解决 vs 未解决: ${avg_per_call_resolved:.6f} vs ${avg_per_call_unresolved:.6f}")
            print(f"    差异: ${diff_per_call:+.6f} ({diff_per_call_pct:+.1f}%)")
        
        # ========== Cost效率分析 ==========
        if resolved_costs:
            print(f"\n💵 Cost 效率分析:")
            resolution_rate = len(resolved_costs) / len(all_costs) * 100
            avg_cost_to_resolve = statistics.mean(resolved_costs)
            total_cost = sum(all_costs)
            total_resolved_cost = sum(resolved_costs)
            
            print(f"  解决率:             {resolution_rate:.1f}%")
            print(f"  解决单个问题成本:   ${avg_cost_to_resolve:.4f}")
            print(f"  成本利用率:         {total_resolved_cost/total_cost*100:.1f}% (${total_resolved_cost:.4f}/${total_cost:.4f})")
            
            if len(resolved_costs) > 0:
                cost_per_resolution = total_cost / len(resolved_costs)
                print(f"  平均解决成本:       ${cost_per_resolution:.4f} (考虑失败尝试)")
    
    print("\n" + "="*80)

In [92]:
analyze_cost_statistics(results)

各Task ID的 Cost 统计分析

【cam-mini-reasoning-medium-res-api】
--------------------------------------------------------------------------------

📊 总体 Cost 统计 (样本数: 50)
  总成本:    $1.0987
  平均值:    $0.0220
  中位数:    $0.0163
  最小值:    $0.0030
  最大值:    $0.1067
  标准差:    $0.0169
  25%分位:   $0.0119
  75%分位:   $0.0288

✅ 已解决问题 Cost 统计 (样本数: 25)
  总成本:    $0.4180
  平均值:    $0.0167
  中位数:    $0.0126
  最小值:    $0.0063
  最大值:    $0.0536
  标准差:    $0.0106

❌ 未解决问题 Cost 统计 (样本数: 25)
  总成本:    $0.6807
  平均值:    $0.0272
  中位数:    $0.0203
  最小值:    $0.0030
  最大值:    $0.1067
  标准差:    $0.0203

⚖️  已解决 vs 未解决 对比:
  平均成本:  $0.0167 vs $0.0272
  差异:      $-0.0105 (-38.6%)
  总成本:    $0.4180 vs $0.6807
  💡 解决问题平均成本更低

🔄 每次 API 调用的平均 Cost 统计

  总体 (样本数: 50):
    平均值:    $0.001889
    中位数:    $0.001813
    最小值:    $0.000980
    最大值:    $0.003588
    标准差:    $0.000542

  已解决问题 (样本数: 25):
    平均值:    $0.001792
    中位数:    $0.001705
    最小值:    $0.000980
    最大值:    $0.002678

  未解决问题 (样本数: 25):
    平均值:    $0.001987


In [32]:
def print_trajectory(logs_dict, task_id, instance_id, max_content_length=500):
    """
    以简洁直观的方式打印指定task_id和instance_id的轨迹日志
    
    参数:
        logs_dict: 日志字典
        task_id: 任务ID
        instance_id: 实例ID
        max_content_length: 内容最大显示长度，超过则截断
    """
    # 检查是否存在
    if task_id not in logs_dict:
        print(f"❌ Task ID '{task_id}' 不存在")
        return
    
    if instance_id not in logs_dict[task_id]:
        print(f"❌ Instance ID '{instance_id}' 在 Task '{task_id}' 中不存在")
        return
    
    instance_data = logs_dict[task_id][instance_id]
    traj = instance_data.get('traj', [])
    is_resolved = instance_data.get('is_resolved', False)
    patch = instance_data.get('patch', '')
    cost = instance_data.get('cost', 0)
    api_calls = instance_data.get('api_calls', 0)
    
    # 打印头部信息
    print("\n" + "="*100)
    print(f"📋 Task: {task_id} | Instance: {instance_id}")
    print("="*100)
    
    status_icon = "✅" if is_resolved else "❌"
    print(f"\n{status_icon} 状态: {'已解决' if is_resolved else '未解决'}")
    print(f"💰 总成本: ${cost:.4f}")
    print(f"📞 API调用次数: {api_calls}")
    print(f"📝 轨迹步骤数: {len(traj)}")
    
    # 累计token统计（包含reasoning tokens）
    total_prompt_tokens = sum(step.get('prompt_tokens', 0) for step in traj)
    total_completion_tokens = sum(step.get('completion_tokens', 0) for step in traj)
    total_reasoning_tokens = sum(step.get('reasoning_tokens', 0) for step in traj)
    total_tokens = sum(step.get('total_tokens', 0) for step in traj)
    
    print(f"🎯 总Token使用: {total_tokens:,}")
    print(f"   ├─ 输入 (input): {total_prompt_tokens:,}")
    print(f"   ├─ 输出 (output): {total_completion_tokens:,}")
    if total_reasoning_tokens > 0:
        print(f"   └─ 推理 (reasoning): {total_reasoning_tokens:,} 🧠")
    
    if not traj:
        print("\n⚠️  没有轨迹记录")
        return
    
    # 打印轨迹
    print("\n" + "-"*100)
    print("📜 轨迹日志")
    print("-"*100)
    
    cumulative_tokens = 0
    cumulative_reasoning_tokens = 0
    
    for i, step in enumerate(traj, 1):
        role = step.get('role', 'unknown')
        content = step.get('content', '')
        prompt_tokens = step.get('prompt_tokens', 0)
        completion_tokens = step.get('completion_tokens', 0)
        reasoning_tokens = step.get('reasoning_tokens', 0)
        step_total_tokens = step.get('total_tokens', 0)
        
        cumulative_tokens += step_total_tokens
        cumulative_reasoning_tokens += reasoning_tokens
        
        # 角色标识
        if role.lower() in ['user', 'human']:
            role_icon = "👤"
            role_label = "USER"
        elif role.lower() in ['assistant', 'agent']:
            role_icon = "🤖"
            role_label = "AGENT"
        elif role.lower() == 'system':
            role_icon = "⚙️"
            role_label = "SYSTEM"
        else:
            role_icon = "❓"
            role_label = role.upper()
        
        # 打印步骤头部
        print(f"\n[步骤 {i}/{len(traj)}] {role_icon} {role_label}")
        
        # Token 信息（根据是否有 reasoning tokens 调整显示）
        if reasoning_tokens > 0:
            print(f"Token: {step_total_tokens:,} (输入: {prompt_tokens:,}, 输出: {completion_tokens:,}, 推理🧠: {reasoning_tokens:,})")
            print(f"累计: {cumulative_tokens:,} tokens (其中推理: {cumulative_reasoning_tokens:,})")
        else:
            print(f"Token: {step_total_tokens:,} (输入: {prompt_tokens:,}, 输出: {completion_tokens:,})")
            print(f"累计: {cumulative_tokens:,} tokens")
        
        print("─" * 100)
        
        # 处理内容
        if len(content) > max_content_length:
            display_content = content[:max_content_length] + f"\n... [截断，共 {len(content)} 字符] ..."
        else:
            display_content = content
        
        # 打印内容（带缩进）
        for line in display_content.split('\n'):
            print(f"  {line}")
    
    # 打印补丁信息
    if patch:
        print("\n" + "-"*100)
        print("🔧 生成的补丁 (Patch)")
        print("-"*100)
        
        patch_lines = patch.split('\n')
        if len(patch_lines) > 30:
            # 只显示前20行和后10行
            print('\n'.join(patch_lines[:20]))
            print(f"\n... [省略 {len(patch_lines) - 30} 行] ...\n")
            print('\n'.join(patch_lines[-10:]))
        else:
            print(patch)
    
    # 打印底部统计
    print("\n" + "="*100)
    print(f"📊 统计摘要:")
    print(f"  • 步骤总数: {len(traj)}")
    print(f"  • 总Token使用: {total_tokens:,}")
    print(f"    - 输入Token: {total_prompt_tokens:,}")
    print(f"    - 输出Token: {total_completion_tokens:,}")
    if total_reasoning_tokens > 0:
        print(f"    - 推理Token: {total_reasoning_tokens:,} 🧠")
        reasoning_ratio = (total_reasoning_tokens / total_tokens * 100) if total_tokens > 0 else 0
        print(f"    - 推理占比: {reasoning_ratio:.1f}%")
    print(f"  • 平均每步Token: {total_tokens/len(traj):.0f}" if traj else "  • 无数据")
    print(f"  • 平均每步成本: ${cost/len(traj):.6f}" if traj else "  • 无数据")
    print(f"  • 结果: {status_icon} {'已解决' if is_resolved else '未解决'}")
    print("="*100 + "\n")


def print_trajectory_compact(logs_dict, task_id, instance_id):
    """
    更紧凑的轨迹打印版本，适合快速浏览
    """
    if task_id not in logs_dict or instance_id not in logs_dict[task_id]:
        print(f"❌ 未找到 {task_id}/{instance_id}")
        return
    
    instance_data = logs_dict[task_id][instance_id]
    traj = instance_data.get('traj', [])
    is_resolved = instance_data.get('is_resolved', False)
    cost = instance_data.get('cost', 0)
    
    # 统计总 token 和 reasoning token
    total_tokens = sum(step.get('total_tokens', 0) for step in traj)
    total_reasoning = sum(step.get('reasoning_tokens', 0) for step in traj)
    
    status = "✅" if is_resolved else "❌"
    
    # 根据是否有 reasoning tokens 调整显示
    if total_reasoning > 0:
        print(f"\n{status} {task_id}/{instance_id} | ${cost:.4f} | {len(traj)} 步 | {total_tokens:,}t (🧠{total_reasoning:,}t)")
    else:
        print(f"\n{status} {task_id}/{instance_id} | ${cost:.4f} | {len(traj)} 步 | {total_tokens:,}t")
    
    print("─" * 90)
    
    for i, step in enumerate(traj, 1):
        role = step.get('role', 'unknown')
        content = step.get('content', '')[:100].replace('\n', ' ')
        tokens = step.get('total_tokens', 0)
        reasoning = step.get('reasoning_tokens', 0)
        
        role_icon = "👤" if 'user' in role.lower() else "🤖"
        
        # 根据是否有 reasoning tokens 调整显示
        if reasoning > 0:
            print(f"{i:2d}. {role_icon} [{tokens:5,}t, 🧠{reasoning:4,}t] {content}...")
        else:
            print(f"{i:2d}. {role_icon} [{tokens:5,}t] {content}...")


def print_trajectory_summary(logs_dict, task_id, instance_id):
    """
    只打印轨迹摘要，不显示详细内容
    """
    if task_id not in logs_dict or instance_id not in logs_dict[task_id]:
        print(f"❌ 未找到 {task_id}/{instance_id}")
        return
    
    instance_data = logs_dict[task_id][instance_id]
    traj = instance_data.get('traj', [])
    is_resolved = instance_data.get('is_resolved', False)
    cost = instance_data.get('cost', 0)
    api_calls = instance_data.get('api_calls', 0)
    
    # 统计所有类型的 tokens
    total_prompt_tokens = sum(step.get('prompt_tokens', 0) for step in traj)
    total_completion_tokens = sum(step.get('completion_tokens', 0) for step in traj)
    total_reasoning_tokens = sum(step.get('reasoning_tokens', 0) for step in traj)
    total_tokens = sum(step.get('total_tokens', 0) for step in traj)
    
    print(f"\n{'='*70}")
    print(f"Task: {task_id} | Instance: {instance_id}")
    print(f"{'='*70}")
    print(f"状态: {'✅ 已解决' if is_resolved else '❌ 未解决'}")
    print(f"成本: ${cost:.4f}")
    print(f"API调用: {api_calls} 次")
    print(f"轨迹步骤: {len(traj)} 步")
    print(f"\nToken使用:")
    print(f"  • 总计: {total_tokens:,}")
    print(f"  • 输入 (input): {total_prompt_tokens:,}")
    print(f"  • 输出 (output): {total_completion_tokens:,}")
    if total_reasoning_tokens > 0:
        print(f"  • 推理 (reasoning): {total_reasoning_tokens:,} 🧠")
        reasoning_ratio = (total_reasoning_tokens / total_tokens * 100) if total_tokens > 0 else 0
        print(f"  • 推理占比: {reasoning_ratio:.1f}%")
    
    if traj:
        print(f"\n步骤分布:")
        role_counts = {}
        role_tokens = {}
        role_reasoning = {}
        
        for step in traj:
            role = step.get('role', 'unknown')
            role_counts[role] = role_counts.get(role, 0) + 1
            role_tokens[role] = role_tokens.get(role, 0) + step.get('total_tokens', 0)
            role_reasoning[role] = role_reasoning.get(role, 0) + step.get('reasoning_tokens', 0)
        
        for role, count in role_counts.items():
            tokens = role_tokens[role]
            reasoning = role_reasoning[role]
            if reasoning > 0:
                print(f"  {role}: {count} 次, {tokens:,} tokens (🧠{reasoning:,})")
            else:
                print(f"  {role}: {count} 次, {tokens:,} tokens")
    
    print(f"{'='*70}\n")


def print_token_analysis(logs_dict, task_id, instance_id):
    """
    专门分析 token 使用情况的函数
    """
    if task_id not in logs_dict or instance_id not in logs_dict[task_id]:
        print(f"❌ 未找到 {task_id}/{instance_id}")
        return
    
    instance_data = logs_dict[task_id][instance_id]
    traj = instance_data.get('traj', [])
    cost = instance_data.get('cost', 0)
    
    if not traj:
        print("⚠️ 没有轨迹数据")
        return
    
    print(f"\n{'='*80}")
    print(f"🔍 Token 使用分析: {task_id}/{instance_id}")
    print(f"{'='*80}")
    
    # 按步骤分析
    print("\n📊 按步骤分析:")
    print(f"{'步骤':<6} {'角色':<10} {'总计':<12} {'输入':<12} {'输出':<12} {'推理':<12}")
    print("-" * 80)
    
    for i, step in enumerate(traj, 1):
        role = step.get('role', 'unknown')[:8]
        total = step.get('total_tokens', 0)
        prompt = step.get('prompt_tokens', 0)
        completion = step.get('completion_tokens', 0)
        reasoning = step.get('reasoning_tokens', 0)
        
        reasoning_mark = " 🧠" if reasoning > 0 else ""
        print(f"{i:<6} {role:<10} {total:>10,}t  {prompt:>10,}t  {completion:>10,}t  {reasoning:>10,}t{reasoning_mark}")
    
    # 汇总统计
    total_tokens = sum(step.get('total_tokens', 0) for step in traj)
    total_prompt = sum(step.get('prompt_tokens', 0) for step in traj)
    total_completion = sum(step.get('completion_tokens', 0) for step in traj)
    total_reasoning = sum(step.get('reasoning_tokens', 0) for step in traj)
    
    print("-" * 80)
    print(f"{'总计':<6} {'':<10} {total_tokens:>10,}t  {total_prompt:>10,}t  {total_completion:>10,}t  {total_reasoning:>10,}t")
    
    # 比例分析
    print(f"\n📈 Token 比例:")
    if total_tokens > 0:
        prompt_ratio = (total_prompt / total_tokens * 100)
        completion_ratio = (total_completion / total_tokens * 100)
        reasoning_ratio = (total_reasoning / total_tokens * 100)
        
        print(f"  • 输入占比: {prompt_ratio:.1f}%")
        print(f"  • 输出占比: {completion_ratio:.1f}%")
        if total_reasoning > 0:
            print(f"  • 推理占比: {reasoning_ratio:.1f}% 🧠")
            print(f"  • 推理/输出比: {(total_reasoning/total_completion*100):.1f}%")
    
    # 成本分析
    if cost > 0 and total_tokens > 0:
        print(f"\n💰 成本分析:")
        cost_per_1k = (cost / total_tokens) * 1000
        print(f"  • 总成本: ${cost:.4f}")
        print(f"  • 每1K tokens成本: ${cost_per_1k:.4f}")
        print(f"  • 平均每步成本: ${cost/len(traj):.6f}")
    
    print(f"{'='*80}\n")

In [14]:
from datasets import load_dataset
dataset = load_dataset("MariusHobbhahn/swe-bench-verified-mini")['test']

INFO:numexpr.utils:Note: NumExpr detected 32 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.
C:\Users\17988\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
INFO:datasets:PyTorch version 2.1.1 available.


In [15]:
def print_swe_bench_item(item, max_length=500):
    """
    美观地打印 SWE-bench 数据项
    
    Args:
        item: 数据集中的一条记录
        max_length: 每个字段显示的最大字符数
    """
    separator = "=" * 80
    sub_separator = "-" * 80
    
    print(f"\n{separator}")
    print(f"📋 Instance ID: {item.get('instance_id', 'N/A')}")
    print(separator)
    
    print(f"\n📝 Problem Statement:")
    print(sub_separator)
    problem = item.get('problem_statement', 'N/A')
    if len(problem) > max_length:
        print(f"{problem[:max_length]}...")
    else:
        print(problem)
    
    print(f"\n{sub_separator}")
    print(f"\n🔧 Patch:")
    print(sub_separator)
    patch = item.get('patch', 'N/A')
    if len(patch) > max_length:
        print(f"{patch[:max_length]}...")
    else:
        print(patch)
    
    print(f"\n{separator}\n")


In [16]:
for i in dataset:
    print_swe_bench_item(i, max_length=100000)


📋 Instance ID: django__django-11790

📝 Problem Statement:
--------------------------------------------------------------------------------
AuthenticationForm's username field doesn't set maxlength HTML attribute.
Description
	
AuthenticationForm's username field doesn't render with maxlength HTML attribute anymore.
Regression introduced in #27515 and 5ceaf14686ce626404afb6a5fbd3d8286410bf13.
​https://groups.google.com/forum/?utm_source=digest&utm_medium=email#!topic/django-developers/qnfSqro0DlA
​https://forum.djangoproject.com/t/possible-authenticationform-max-length-regression-in-django-2-1/241


--------------------------------------------------------------------------------

🔧 Patch:
--------------------------------------------------------------------------------
diff --git a/django/contrib/auth/forms.py b/django/contrib/auth/forms.py
--- a/django/contrib/auth/forms.py
+++ b/django/contrib/auth/forms.py
@@ -191,7 +191,9 @@ def __init__(self, request=None, *args, **kwargs):
 
     

In [17]:
# example：django__django-11999
# 11815
# print_trajectory(results, 'cam-output', 'sphinx-doc__sphinx-10435', max_content_length=10000)

In [187]:
print_trajectory(results, 'mini-reasoning-high-res-api', 'django__django-12276', max_content_length=10000)


📋 Task: mini-reasoning-high-res-api | Instance: django__django-12276

✅ 状态: 已解决
💰 总成本: $0.0456
📞 API调用次数: 14
📝 轨迹步骤数: 30
🎯 总Token使用: 161,133
   ├─ 输入 (input): 142,594
   ├─ 输出 (output): 18,539
   └─ 推理 (reasoning): 16,768 🧠

----------------------------------------------------------------------------------------------------
📜 轨迹日志
----------------------------------------------------------------------------------------------------

[步骤 1/30] ⚙️ SYSTEM
Token: 0 (输入: 0, 输出: 0)
累计: 0 tokens
────────────────────────────────────────────────────────────────────────────────────────────────────
  You are a helpful assistant that can interact multiple times with a computer shell to solve programming tasks.
  Your response must contain exactly ONE bash code block with ONE command (or commands connected with && or ||).
  
  Include a THOUGHT section before your command where you explain your reasoning process.
  Format your response as shown in <format_example>.
  
  <format_example>
  THOUGHT: Y

In [188]:
print_trajectory(results, 'mini-first-high-reasoning-4', 'django__django-12276', max_content_length=10000)


📋 Task: mini-first-high-reasoning-4 | Instance: django__django-12276

✅ 状态: 已解决
💰 总成本: $0.0189
📞 API调用次数: 22
📝 轨迹步骤数: 46
🎯 总Token使用: 238,480
   ├─ 输入 (input): 234,397
   ├─ 输出 (output): 4,083
   └─ 推理 (reasoning): 1,536 🧠

----------------------------------------------------------------------------------------------------
📜 轨迹日志
----------------------------------------------------------------------------------------------------

[步骤 1/46] ⚙️ SYSTEM
Token: 0 (输入: 0, 输出: 0)
累计: 0 tokens
────────────────────────────────────────────────────────────────────────────────────────────────────
  You are a helpful assistant that can interact multiple times with a computer shell to solve programming tasks.
  Your response must contain exactly ONE bash code block with ONE command (or commands connected with && or ||).
  
  Include a THOUGHT section before your command where you explain your reasoning process.
  Format your response as shown in <format_example>.
  
  <format_example>
  THOUGHT: You

In [189]:
print_trajectory(results, 'mini-reasoning-low-res-api', 'django__django-12276', max_content_length=10000)


📋 Task: mini-reasoning-low-res-api | Instance: django__django-12276

❌ 状态: 未解决
💰 总成本: $0.0093
📞 API调用次数: 10
📝 轨迹步骤数: 22
🎯 总Token使用: 76,065
   ├─ 输入 (input): 74,710
   ├─ 输出 (output): 1,355
   └─ 推理 (reasoning): 576 🧠

----------------------------------------------------------------------------------------------------
📜 轨迹日志
----------------------------------------------------------------------------------------------------

[步骤 1/22] ⚙️ SYSTEM
Token: 0 (输入: 0, 输出: 0)
累计: 0 tokens
────────────────────────────────────────────────────────────────────────────────────────────────────
  You are a helpful assistant that can interact multiple times with a computer shell to solve programming tasks.
  Your response must contain exactly ONE bash code block with ONE command (or commands connected with && or ||).
  
  Include a THOUGHT section before your command where you explain your reasoning process.
  Format your response as shown in <format_example>.
  
  <format_example>
  THOUGHT: Your rea

In [74]:
# set([i for i,v in results['mini-reasoning-medium-res-api'].items() if v['is_resolved']])-set([i for i,v in results['cam-mini-reasoning-medium-res-api'].items() if v['is_resolved']])

In [180]:
set([i for i,v in results['mini-reasoning-high-res-api'].items() if v['is_resolved']]) - set([i for i,v in results['mini-first-high-reasoning-12'].items() if v['is_resolved']])

{'sphinx-doc__sphinx-10673', 'sphinx-doc__sphinx-9230'}

In [171]:
set([i for i,v in results['mini-reasoning-high-res-api'].items() if v['is_resolved']]) - set([i for i,v in results['mini-first-high-reasoning-4'].items() if v['is_resolved']])

{'sphinx-doc__sphinx-7757', 'sphinx-doc__sphinx-8035'}

In [177]:
set([i for i,v in results['mini-first-high-reasoning-4'].items() if v['is_resolved']]) - set([i for i,v in results['mini-reasoning-low-res-api'].items() if v['is_resolved']])

{'django__django-12193',
 'django__django-12276',
 'django__django-12304',
 'django__django-12708',
 'django__django-12774',
 'sphinx-doc__sphinx-10673',
 'sphinx-doc__sphinx-8551',
 'sphinx-doc__sphinx-9230'}

In [113]:
print_trajectory(results, 'cam-mini-reasoning-high', 'django__django-12039', max_content_length=10000)


📋 Task: cam-mini-reasoning-high | Instance: django__django-12039

✅ 状态: 已解决
💰 总成本: $0.0404
📞 API调用次数: 4
📝 轨迹步骤数: 10
🎯 总Token使用: 37,047
   ├─ 输入 (input): 17,144
   ├─ 输出 (output): 19,903
   └─ 推理 (reasoning): 18,048 🧠

----------------------------------------------------------------------------------------------------
📜 轨迹日志
----------------------------------------------------------------------------------------------------

[步骤 1/10] ⚙️ SYSTEM
Token: 0 (输入: 0, 输出: 0)
累计: 0 tokens
────────────────────────────────────────────────────────────────────────────────────────────────────
  You are an ultra token-efficient Automated Program Repair (APR) Agent. Your goal is to fix bugs via a State-Action-Return loop with minimal token usages. Each round of your output is one python code style message formated as follows.
  
  <format_example>
  # <Reasoning Summary>
  @<stage>
  def <action>_to_<intent>():
      ```bash
      cmd
      ```
  </format_example>
  
  Format Description:
  0. Commen

In [52]:
import re

s="""@bug_reproduction
def grep_enum_occurrences_to_find_relevant_files():
    ```bash
    grep -R --line-number "Enum" . || true
    ```"""

# 修改后的正则：
# 1. \s*```bash : 允许开头有空格
# 2. \n\s*```   : 允许结尾的 ``` 前面有空格
action_regex = r"\s*```bash\s*\n(.*?)\n\s*```"

# 输出结果
print(re.findall(action_regex, s, re.DOTALL))

['    grep -R --line-number "Enum" . || true']
